In [ ]:
import git
import pandas as pd




pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
pd.read_csv("")
pd.read_csv("")



repoLocal = git.Repo( 'C:/Users/limc_/Documents/GitHub/Datos' )
print(repoLocal.git.status())
repoLocal.git.add(".")
try:
    repoLocal.git.commit(m='Update automatico via python')
except:
    pass

origin = repoLocal.remote(name='origin')
origin.push()